<a href="https://colab.research.google.com/github/HPTHECONQUEROR/Centillion_labs_updates/blob/main/LLaVA_13b_4bit_vanilla_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %cd /content
# !git clone -b v1.0 https://github.com/camenduru/LLaVA
# %cd /content/LLaVA

# !pip install -q transformers==4.36.2
# !pip install -q gradio .

from transformers import AutoTokenizer, BitsAndBytesConfig
from llava.model import LlavaLlamaForCausalLM
import torch

model_path = "4bit/llava-v1.5-13b-3GB"
kwargs = {"device_map": "auto"}
kwargs['load_in_4bit'] = True
kwargs['quantization_config'] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)
model = LlavaLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True, **kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

vision_tower = model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower.to(device='cuda')
image_processor = vision_tower.image_processor

import os
import requests
from PIL import Image
from io import BytesIO
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from transformers import TextStreamer

def caption_image(image_file, prompt):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    disable_torch_init()
    conv_mode = "llava_v0"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
    inp = f"{roles[0]}: {prompt}"
    inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    raw_prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(raw_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
    with torch.inference_mode():
      output_ids = model.generate(input_ids, images=image_tensor, do_sample=True, temperature=0.2,
                                  max_new_tokens=1024, use_cache=True, stopping_criteria=[stopping_criteria])
    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    conv.messages[-1][-1] = outputs
    output = outputs.rsplit('</s>', 1)[0]
    return image, output

[2025-04-21 17:06:48,874] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

pytorch_model-00001-of-00009.bin:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [3]:
!pip install numpy==1.26.4


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.21 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy

In [3]:
prompt = """

You're given an image of one or more graphs (any type: line, bar, pie, area, scatter, etc.).

Your job is to:

Identify each graph type automatically
Extract all plotted data points accurately
Output everything in a clean, structured JSON format (as shown below)

__Output Format__
{
  "infographics": [
    {
      "type": "graph_type",  // e.g., line_graph, bar_chart, scatter_plot
      "title": "",
      "position": ["x1", "y1", "x2", "y2"],
      "xAxis": [],  // All x-axis labels
      "yAxis": [],  // All y-axis labels or scale
      "data_table": [
        { "x": "", "Series 1": value, "Series 2": value } // actual values
      ],
      "plots": [
        {
          "label": "Series 1",
          "points": [[x, y], [x, y], ...],  // All plotted values
          "behavior": "increasing | decreasing | fluctuating | stable",
          "highest_point": [x, y],
          "lowest_point": [x, y]
        }
        // Repeat for each series/plot
      ]
    }
    // Repeat for multiple graphs if present
  ]
}

__Key Instructions__
Work for any graph type: bar, line, area, scatter, pie, etc.

Extract actual data points only (not axis ticks or grid labels).

Include x/y labels, title, and plotted values.

Include table form of the graph data (data_table).

Mention behavior of each series + highest/lowest (x, y) points.

Be strict with structure and accuracy.
"""



image, output = caption_image(f'/content/graph_11.jpg', prompt)
print(output)

image, output = caption_image(f'/content/graph_26.jpg', prompt)
print(output)

image, output = caption_image(f'/content/graph_32.jpg', prompt)
print(output)

image, output = caption_image(f'/content/graph_6.jpg', prompt)
print(output)




{
"infographics": [
{
"type": "line\_graph",
"title": "Profitability Remains But Squeezing",
"position": ["Q1", "Q2", "Q3", "Q4"],
"xAxis": ["Quarter"],
"yAxis": ["Profitability"],
"data\_table": [
{
"x": "Quarter",
"Series 1": "Profitability",
"Series 2": ""
}
],
"plots": [
{
"label": "Profitability",
"points": [
[0.0, 0.62],
[0.25, 0.62],
[0.5, 0.62],
[0.75, 0.62]
],
"behavior": "stable",
"highest\_point": [0.5, 0.62],
"lowest\_point": [0.0, 0.62]
}
]
}
]
{
"infographics": [
{
"type": "line\_graph",
"title": "Smartphone Drops % QoQ for Seasonality Q1 2014",
"position": ["Q1", "QoQ"],
"xAxis": ["Q1", "Q2", "Q3", "Q4"],
"yAxis": ["% QoQ"],
"data\_table": [
{ "x": "Q1", "Series 1": "-2.3", "Series 2": "-2.3" },
{ "x": "Q2", "Series 1": "-1.6", "Series 2": "-1.6" },
{ "x": "Q3", "Series 1": "-1.1", "Series 2": "-1.1" },
{ "x": "Q4", "Series 1": "-0.8", "Series 2": "-0.8" }
],
"plots": [
{
"label": "Smartphone Drops % QoQ",
"points": [
[0.0, -2.3],
[0.25, -2.3],
[0.5, -1.6],
[0.75, -1.1],

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content
!git clone -b v1.0 https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install -q gradio .

/content
Cloning into 'LLaVA'...
remote: Enumerating objects: 1960, done.
remote: Total 1960 (delta 0), reused 0 (delta 0), pack-reused 1960 (from 1)
Receiving objects: 100% (1960/1960), 13.60 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.
/content/LLaVA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB